# Vorbereitende Schritte

In [30]:
!pip install dash pandas plotly

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
from statistics import mean
import glob
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba
from matplotlib.ticker import FuncFormatter
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import math
import gdown

<ipython-input-2-e6636834bc97>:14: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-2-e6636834bc97>:15: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [24]:
# Daten einlesen
#dateipfad = '/content/drive/MyDrive/DIIHK/Data/gesamter_Datensatz_nach_Land_sortiert.csv'
#all_sorted = pd.read_csv(dateipfad, encoding='utf-8')

# https://drive.google.com/file/d/1-Qn9Zg4sxmAwhxsATyJNQ--EQZdqAGS8/view?usp=sharing
#url = "https://drive.google.com/uc?id=1-Qn9Zg4sxmAwhxsATyJNQ--EQZdqAGS8"
#all_sorted = pd.read_csv(url, encoding='utf-8')



# Sicherstellen, dass die Zeitraum-Spalte als Datum erkannt wird
#all_sorted['Zeitraum'] = pd.to_datetime(all_sorted['Zeitraum'])
#all_sorted['Jahr'] = all_sorted['Zeitraum'].dt.year

KeyError: 'Zeitraum'

In [31]:
# Verwende die ID der Datei aus der Google Drive URL
file_id = '1-Qn9Zg4sxmAwhxsATyJNQ--EQZdqAGS8'
url = f"https://drive.google.com/uc?id={file_id}"

# Die Datei herunterladen
gdown.download(url, 'gesamter_Datensatz_nach_Land_sortiert.csv', quiet=False)

# Danach die CSV-Datei in Pandas laden
all_sorted = pd.read_csv('gesamter_Datensatz_nach_Land_sortiert.csv', encoding='utf-8')

# Sicherstellen, dass die Zeitraum-Spalte als Datum erkannt wird
all_sorted['Zeitraum'] = pd.to_datetime(all_sorted['Zeitraum'])
all_sorted['Jahr'] = all_sorted['Zeitraum'].dt.year

Downloading...
From (original): https://drive.google.com/uc?id=1-Qn9Zg4sxmAwhxsATyJNQ--EQZdqAGS8
From (redirected): https://drive.google.com/uc?id=1-Qn9Zg4sxmAwhxsATyJNQ--EQZdqAGS8&confirm=t&uuid=30a28c08-b749-471f-a355-6eb209769e55
To: /content/gesamter_Datensatz_nach_Land_sortiert.csv
100%|██████████| 461M/461M [00:11<00:00, 39.4MB/s]


In [32]:
#all_sorted

,Land,Zeitraum,Code,Label,Ausfuhr: Gewicht,Ausfuhr: Wert,Ausfuhr: Wert (US-Dollar),Einfuhr: Gewicht,Einfuhr: Wert,Einfuhr: Wert (US-Dollar),Jahr
0,Afghanistan,2008-01-31,WA01,Lebende Tiere,0.0,0,0,0.0,0,0,2008
1,Afghanistan,2008-01-31,WA02,Fleisch,17.3,63000,93000,0.0,0,0,2008
2,Afghanistan,2008-01-31,WA03,"Fische, Krebs- und Weichtiere",0.8,4000,5000,0.0,0,0,2008
3,Afghanistan,2008-01-31,WA04,"Milch, Milcherzeugnisse, Eier, Honig",338.8,319000,469000,0.0,0,0,2008
4,Afghanistan,2008-01-31,WA05,Andere Waren tierischen Ursprungs,0.0,0,0,0.0,0,0,2008
...,...,...,...,...,...,...,...,...,...,...,...
5237899,Österreich,2024-12-31,WA95,"Spielzeug, Spiele usw., Sportgeräte",0.0,64891000,67999000,0.0,8420000,8823000,2024
5237900,Österreich,2024-12-31,WA96,Verschiedene Waren,0.0,21633000,22669000,0.0,4986000,5225000,2024
5237901,Österreich,2024-12-31,WA97,"Kunstgegenstände, Sammlungsstücke",0.0,1433000,1501000,0.0,45000,47000,2024
5237902,Österreich,2024-12-31,WA98,Vollständige Fabrikationsanlagen,0.0,0,0,0.0,0,0,2024


In [33]:
### Each year's export, import and trade volume for each country + wachstum + differenz + ranking

# Gruppieren nach Land und Jahr
all_sorted['Jahr'] = all_sorted['Zeitraum'].dt.year
all_sorted['Monat'] = all_sorted['Zeitraum'].dt.month

# Berechnungen für Export, Import und Handelsvolumen
df_grouped = all_sorted.groupby(['Land', 'Jahr']).agg(
    export_wert=('Ausfuhr: Wert', 'sum'),
    import_wert=('Einfuhr: Wert', 'sum')
).reset_index()

# Berechnen des Handelsvolumenwerts
df_grouped['handelsvolumen_wert'] = df_grouped['export_wert'] + df_grouped['import_wert']

# Berechnen der Handelsbilanz
df_grouped['handelsbilanz'] = df_grouped['export_wert'] - df_grouped['import_wert']

# Definieren ob Überschuss oder Defizit
df_grouped['handelsbilanz_status'] = df_grouped['handelsbilanz'].apply(lambda x: 'Überschuss' if x > 0 else 'Defizit')

# Ranking für jedes Jahr nach den 3 Kategorien (Export, Import, Handelsvolumen)
df_grouped['export_ranking'] = df_grouped.groupby('Jahr')['export_wert'].rank(ascending=False)
df_grouped['import_ranking'] = df_grouped.groupby('Jahr')['import_wert'].rank(ascending=False)
df_grouped['handelsvolumen_ranking'] = df_grouped.groupby('Jahr')['handelsvolumen_wert'].rank(ascending=False)

# Top 10 Länder nach Jahr und Kategorie
top_10_export = df_grouped[df_grouped['export_ranking'] <= 10]
top_10_import = df_grouped[df_grouped['import_ranking'] <= 10]
top_10_handelsvolumen = df_grouped[df_grouped['handelsvolumen_ranking'] <= 10]

# Berechnen des Wachstums für Export, Import und Handelsvolumen
df_grouped['export_wachstum'] = df_grouped.groupby('Land')['export_wert'].pct_change() * 100
df_grouped['import_wachstum'] = df_grouped.groupby('Land')['import_wert'].pct_change() * 100
df_grouped['handelsvolumen_wachstum'] = df_grouped.groupby('Land')['handelsvolumen_wert'].pct_change() * 100

# Für 2008 setzen wir das Wachstum auf 0, da es kein Vorjahr gibt
df_grouped.loc[df_grouped['Jahr'] == 2008, 'export_wachstum'] = 0
df_grouped.loc[df_grouped['Jahr'] == 2008, 'import_wachstum'] = 0
df_grouped.loc[df_grouped['Jahr'] == 2008, 'handelsvolumen_wachstum'] = 0

# Berechnen des Rankings für das Wachstum in jeder Kategorie
df_grouped['export_wachstum_ranking'] = df_grouped.groupby('Jahr')['export_wachstum'].rank(ascending=False)
df_grouped['import_wachstum_ranking'] = df_grouped.groupby('Jahr')['import_wachstum'].rank(ascending=False)
df_grouped['handelsvolumen_wachstum_ranking'] = df_grouped.groupby('Jahr')['handelsvolumen_wachstum'].rank(ascending=False)

# Berechnung der absoluten Differenzen zum Vorjahr
df_grouped['export_differenz'] = df_grouped.groupby('Land')['export_wert'].diff()
df_grouped['import_differenz'] = df_grouped.groupby('Land')['import_wert'].diff()
df_grouped['handelsvolumen_differenz'] = df_grouped.groupby('Land')['handelsvolumen_wert'].diff()

# Für 2008 setzen wir die Differenzen auf 0, da es kein Vorjahr gibt
df_grouped.loc[df_grouped['Jahr'] == 2008, 'export_differenz'] = 0
df_grouped.loc[df_grouped['Jahr'] == 2008, 'import_differenz'] = 0
df_grouped.loc[df_grouped['Jahr'] == 2008, 'handelsvolumen_differenz'] = 0

# Ausgabe des DataFrames mit den neuen Spalten
df_grouped.head()

,Land,Jahr,export_wert,import_wert,handelsvolumen_wert,handelsbilanz,handelsbilanz_status,export_ranking,import_ranking,handelsvolumen_ranking,export_wachstum,import_wachstum,handelsvolumen_wachstum,export_wachstum_ranking,import_wachstum_ranking,handelsvolumen_wachstum_ranking,export_differenz,import_differenz,handelsvolumen_differenz
0,Afghanistan,2008,267487000,2763000,270250000,264724000,Überschuss,92.0,170.0,102.0,0.000000,0.000000,0.000000,131.5,131.5,131.5,0.0,0.0,0.0
1,Afghanistan,2009,224888000,4239000,229127000,220649000,Überschuss,91.0,160.0,106.0,-15.925634,53.420195,-15.216651,123.0,38.0,117.0,-42599000.0,1476000.0,-41123000.0
2,Afghanistan,2010,268612000,23186000,291798000,245426000,Überschuss,92.0,136.0,105.0,19.442567,446.968625,27.352080,103.0,13.0,70.0,43724000.0,18947000.0,62671000.0
3,Afghanistan,2011,282486000,10883000,293369000,271603000,Überschuss,95.0,151.0,112.0,5.165071,-53.062193,0.538386,140.0,202.0,169.0,13874000.0,-12303000.0,1571000.0
4,Afghanistan,2012,294068000,14254000,308322000,279814000,Überschuss,96.0,144.0,108.0,4.100026,30.974915,5.096994,132.0,41.0,105.0,11582000.0,3371000.0,14953000.0


In [19]:
#!pip freeze > requirements.txt
#from google.colab import files
#files.download('requirements.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Überblick über Deutschlands Handel

## Gesamtüberblick seit 2008 bis 2024

### ✅ Gesamter Export-, Import- und Handelsvolumen-Verlauf Deutschlands

In [34]:
# Funktion zur Formatierung der Y-Achse
def formatter(value):
    if value >= 1e9:
        return f'{value / 1e9:.0f} Mrd'
    elif value >= 1e6:
        return f'{value / 1e6:.0f} Mio'
    elif value >= 1e3:
        return f'{value / 1e3:.0f} K'
    else:
        return str(value)


# Funktion zur Formatierung der Y-Achse
# def formatter(value):
#     if value >= 1e9:
#         return f'{value / 1e9:.1f} Mrd'
#     elif value >= 1e6:
#         return f'{value / 1e6:.1f} Mio'
#     elif value >= 1e3:
#         return f'{value / 1e3:.1f} K'
#     else:
#         return str(value)

# Gruppierung der Daten nach Jahr
gesamt_deutschland = all_sorted.groupby('Jahr').agg(
    gesamt_export=('Ausfuhr: Wert', 'sum'),
    gesamt_import=('Einfuhr: Wert', 'sum')
).reset_index()

gesamt_deutschland['gesamt_handelsvolumen'] = gesamt_deutschland['gesamt_export'] + gesamt_deutschland['gesamt_import']

# Dash-App erstellen
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("Deutschlands Handelsentwicklung"),
    dcc.Graph(id='handel_graph'),
])

@app.callback(
    Output('handel_graph', 'figure'),
    Input('handel_graph', 'id')
)
def update_graph(_):
    fig = go.Figure()

    # Linien für Export, Import und Handelsvolumen
    for col, name, color in zip(
        ['gesamt_export', 'gesamt_import', 'gesamt_handelsvolumen'],
        ['Exportvolumen', 'Importvolumen', 'Gesamthandelsvolumen'],
        ['#1f77b4', '#ff7f0e', '#2ca02c']
    ):
        fig.add_trace(go.Scatter(
            x=gesamt_deutschland['Jahr'],
            y=gesamt_deutschland[col],
            mode='lines+markers',
            name=name,
            line=dict(width=2, color=color),
            hovertemplate=f'<b>{name}</b><br>Jahr: %{{x}}<br>Wert: %{{y:,.0f}} €'
        ))

    # Berechnung der maximalen Y-Achse, um die Schrittgröße zu bestimmen
    max_value = max(gesamt_deutschland['gesamt_export'].max(),
                    gesamt_deutschland['gesamt_import'].max(),
                    gesamt_deutschland['gesamt_handelsvolumen'].max())

    # Berechnung der Tick-Schritte basierend auf dem maximalen Wert
    tick_step = 500e9  # 500 Mrd als Standard-Schrittgröße
    tickvals = np.arange(0, max_value + tick_step, tick_step)

    # Layout-Anpassungen
    fig.update_layout(
        title='Entwicklung von Export, Import und Handelsvolumen',
        xaxis_title='Jahr',
        yaxis_title='Wert in €',
        yaxis=dict(
            tickformat=',',
            tickvals=tickvals,
            ticktext=[formatter(val) for val in tickvals]
        ),
        legend=dict(title='Kategorie', bgcolor='rgba(255,255,255,0.7)')
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

## Überblick nach bestimmtem Jahr

### ✅ Monatlicher Handelsverlauf

In [35]:
# Funktion zur Formatierung der Y-Achse
def formatter(value):
    if value >= 1e9:
        return f'{value / 1e9:.0f} Mrd'
    elif value >= 1e6:
        return f'{value / 1e6:.0f} Mio'
    elif value >= 1e3:
        return f'{value / 1e3:.0f} K'
    else:
        return str(value)


# Berechnungen für Export, Import und Handelsvolumen
gesamt_deutschland_monthly = all_sorted.groupby(['Jahr', 'Monat']).agg(
    export_wert=('Ausfuhr: Wert', 'sum'),
    import_wert=('Einfuhr: Wert', 'sum')
).reset_index()

# Berechnen des Handelsvolumenwerts
gesamt_deutschland_monthly['handelsvolumen_wert'] = gesamt_deutschland_monthly['export_wert'] + gesamt_deutschland_monthly['import_wert']


# Dash-App erstellen
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("Monatlicher Handelsverlauf Deutschlands"),

    dcc.Dropdown(
        id='jahr_dropdown',
        options=[{'label': str(j), 'value': j} for j in sorted(gesamt_deutschland_monthly['Jahr'].unique())],
        value=2024,  # Standardwert
        clearable=False,
        style={'width': '50%'}
    ),

    dcc.Graph(id='monatlicher_handel_graph'),
])

@app.callback(
    Output('monatlicher_handel_graph', 'figure'),
    Input('jahr_dropdown', 'value')
)
def update_graph(year_selected):
    df_year_monthly = gesamt_deutschland_monthly[gesamt_deutschland_monthly['Jahr'] == year_selected]

    fig = go.Figure()

    for col, name, color in zip(
        ['export_wert', 'import_wert', 'handelsvolumen_wert'],
        ['Exportvolumen', 'Importvolumen', 'Gesamthandelsvolumen'],
        ['#1f77b4', '#ff7f0e', '#2ca02c']
    ):
        fig.add_trace(go.Scatter(
            x=df_year_monthly['Monat'],
            y=df_year_monthly[col],
            mode='lines+markers',
            name=name,
            line=dict(width=2, color=color),
            hovertemplate=f'<b>{name}</b><br>Monat: %{{x}}<br>Wert: %{{y:,.0f}} €'
        ))

    # Maximale Werte bestimmen
    max_value = df_year_monthly[['export_wert', 'import_wert', 'handelsvolumen_wert']].values.max()

    # Auf nächste 50 Mrd aufrunden
    rounded_max = math.ceil(max_value / 50e9) * 50e9

    # Y-Achse in 50-Mrd-Schritten skalieren
    tickvals = np.arange(0, rounded_max + 1, 50e9)
    ticktext = [formatter(val) for val in tickvals]

    # Achsen und Layout
    fig.update_layout(
        title=f'Monatlicher Export-, Import- und Handelsverlauf Deutschlands im Jahr {year_selected}',
        xaxis_title='Monat',
        yaxis_title='Wert in €',
        xaxis=dict(
            tickmode='array',
            tickvals=list(range(1, 13)),
            ticktext=['Jan', 'Feb', 'Mär', 'Apr', 'Mai', 'Jun', 'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dez']
        ),
        yaxis=dict(
            tickvals=tickvals,
            ticktext=ticktext
        ),
        legend=dict(title='Kategorie', bgcolor='rgba(255,255,255,0.7)')
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>